In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import pandas as pd 
from selenium.webdriver.common.action_chains import ActionChains
import re

In [ ]:
# 유튜브 기본 url - 채널 href 없는 경우, 영상 다이렉트링크에 붙여넣을 때 사용
base_url = 'https://www.youtube.com'

# 검색창에 # 붙인 상태
url = 'https://www.youtube.com/results?search_query=%23'

# 검색 필터(동영상만 보이기, 관련성 기준) 추가 url
last_url = '&sp=EgIQAQ%253D%253D'

# 검색어
keyword = input()

# '# 검색어' 최종 url
search = url+keyword+last_url

# 데이터 들어갈 빈 리스트
urls = []
titles = []
thumbnails = []
host_urls = []
youtube_url = []
youtube_thumbnail = []
youtube_title = []
youtube_host_url = []
youtube_avatar = []
collect = []

df_url = []
df_title = []
df_host = []
df_host_url = []
df_avatar = []
df_follow = []
df_upload = []
df_view = []
df_thumbnail = []
df_likes = []
df_hates = []
df_comment = []

# 크롬드라이버로 단어 검색하기
driver = webdriver.Chrome('chromedriver.exe')
driver.get(search)
driver.implicitly_wait(5.0)

# 영상 목록 스크롤 10번한 뒤 영상들의 제목 따오기
for i in range(10) :
    ActionChains(driver).send_keys(Keys.END).perform()
    time.sleep(0.5)
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'lxml')
    videos = soup.select('h3 > #video-title')

# 영상 url, 제목, 썸네일, 유튜버채널url 가져오기
# 썸네일, 채널url이 비어있는 경우에는 가상이미지, 유튜브기본url로 대체
for i in range(len(videos)): 
    urls.append(base_url + soup.select('h3 > #video-title')[i]['href'])
    titles.append(soup.select('h3 > #video-title')[i]['title'])
    try:
        thumbnail = soup.select('#dismissable > ytd-thumbnail > #thumbnail > yt-img-shadow > #img')[i]['src']
        thumbnail = thumbnail.replace('&', '&amp;')
        thumbnails.append(thumbnail)
    except:
        thumbnail = 'https://via.placeholder.com/1280x720'
        thumbnails.append(thumbnail)
    try:
        host_url = soup.select('#container > #text-container > #text > a')[i]['href']
        host_urls.append(base_url + host_url)
    except:
        host_urls.append(base_url)
        
driver.close()

# 영상제목이 영어+특수문자인 경우만 필터링
for i in range(len(titles)) :
    if re.match('[!-~·\s]+', titles[i]) == None :
        collect.append(None)
    else :
        if re.match('[!-~\s]+', titles[i]).group() == titles[i] :
            collect.append(titles[i])
        else :
            collect.append(None)

# 영어제목의 영상url, 썸네일, 채널url, 제목만 저장  
for i in range(len(collect)) :
    if collect[i] != None :
        youtube_url.append(urls[i])
        youtube_thumbnail.append(thumbnails[i])
        youtube_host_url.append(host_urls[i])
        youtube_title.append(titles[i])
        
# print(youtube_url)

# 필터링 된 상태의 url로 반복문
for i in range(len(youtube_url)) :
    
    driver = webdriver.Chrome('chromedriver.exe')
    driver.get(youtube_url[i])
    driver.implicitly_wait(5.0)
    time.sleep(3.0)

    # 영어설정 되면 설정하고 안되면 해당영상 건너뛰기
    try :
        driver.find_elements_by_xpath('//*[@id="buttons"]/ytd-topbar-menu-button-renderer[3]')[0].click()
        driver.find_elements_by_xpath('//*[@id="endpoint"]/paper-item')[0].click()
        driver.find_elements_by_xpath('//*[@id="settings"]/ytd-account-settings/div/div[2]/paper-item[13]')[0].click()
    except :
        driver.close()
        continue
    
    time.sleep(3.0)
    
    # 행 갯수를 맞춰서 복붙하기 위해 미리 영상별로 변수에 삽입
    this_url = youtube_url[i]
    this_thumbnail = youtube_thumbnail[i]
    this_host_url = youtube_host_url[i]
    this_title = youtube_title[i]
    
    # 자동스크롤 100번 하기
    # 그냥 END 키만 누르면 영상끝으로 가는 경우도 있어 ctrl + end 키로 작업
    ActionChains(driver).send_keys(Keys.SHIFT+'k').perform()
    time.sleep(1.0)
    for i in range(100) : 
        ActionChains(driver).send_keys(Keys.CONTROL).send_keys(Keys.END).perform()
        time.sleep(0.5)
    
    # 스크롤 완료되면 유튜버이름, 아바타이미지, 구독자수, 영상업로드날짜, 영상별조회수, 댓글 가져오기
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'lxml')
        
    host = soup.select('#text > a')
    try:
        avatar = soup.select('#avatar > #img')[0]['src']
    except:
        avatar = 'https://via.placeholder.com/48x48'
    follow = soup.select('#owner-sub-count')
    upload = soup.select('#date > yt-formatted-string')
    view = soup.select('#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer')
    likeshates = soup.select('#top-level-buttons > ytd-toggle-button-renderer > a > #text')
    comment = soup.select('#content-text')

    # 각 영상별로 댓글수만큼 나머지 컬럼 복붙
    for i in range(len(comment)):

        # url
        df_url.append(this_url)

        # title
        df_title.append(this_title)

        # 호스트 url
        df_host_url.append(this_host_url)
        
        # 사진
        df_avatar.append(avatar)
                
        # 썸네일
        df_thumbnail.append(this_thumbnail)
        
        # 호스트
        str_host = host[0].text
        df_host.append(str_host)
        
        # 구독자
        str_follow = follow[0].text
        df_follow.append(str_follow)

        # 영상 작성일자
        str_upload = upload[0].text
        df_upload.append(str_upload)

        # 재생횟수
        str_view = view[0].text
        df_view.append(str_view)
        
        # 좋아요
        str_likes = likeshates[0].text
        df_likes.append(str_likes)
        # 싫어요
        str_hates = likeshates[1].text
        df_hates.append(str_hates)
        
        # 내용
        str_comment = str(comment[i].text)
        str_comment = str_comment.replace('\n', '')
        str_comment = str_comment.replace('\t','')
        str_comment = str_comment.replace('               ', '')
#         df_comment.append(str_comment)
        
        # 댓글내용 영어만 골라서 다시 저장하기
        m = re.findall("[a-zA-Z']+", str_comment)
        a = ''
        for i in m :
            a = a + ' ' + i
        df_comment.append(a)
        
#     print(this_url)
    driver.close()

# 데이터프레임 만들고 '검색어.csv' 로 저장 
youtube_df = pd.DataFrame({
    "url" : df_url,
    "title" : df_title,
    "host" : df_host,
    "host_url" : df_host_url,
    "avatar" : df_avatar,
    "follow" : df_follow,
    "upload" : df_upload,
    "view" : df_view,
    "thumbnail" : df_thumbnail,
    "likes" : df_likes,
    "hates" : df_hates,
    "comment" : df_comment
})

# SQL에 넣기위해 콤마 제거, 댓글내용중에 영어만 저장
youtube_df['title'] = youtube_df['title'].str.replace(',', ' ')
youtube_df['view'] = youtube_df['view'].str.replace(',', '')
youtube_df['comment'] = youtube_df['comment'].str.replace(',', ' ')
youtube_df.dropna(inplace = True)
print(youtube_df)
youtube_df.to_csv('{}'.format(keyword))